# Лабораторна робота: Основні парадигми ООП у Python

## 1. Інкапсуляція

In [ ]:
import random

class BankAccount:
    def __init__(self, owner, balance):
        self.owner = owner
        self.__balance = balance  # Приватний атрибут

    def deposit(self, amount):
        self.__balance += amount
        return f"Поповнено на {amount}"

    def withdraw(self, amount):
        if amount <= self.__balance:
            self.__balance -= amount
            return f"Знято {amount}"
        return "Недостатньо коштів"

    def get_balance(self):
        return self.__balance

account = BankAccount("Bohdan", 1000)

print(f"Початковий баланс: {account.get_balance()}")
for i in range(5):
    val = random.randint(50, 300)
    action = random.choice([account.deposit, account.withdraw])
    print(f"Операція {i+1}: {action(val)}, Баланс: {account.get_balance()}")

## 2. Наслідування

In [ ]:
class Vehicle:
    def __init__(self, brand, model):
        self.brand = brand
        self.model = model

    def display_info(self):
        return f"{self.brand} {self.model}"
    
    def service_info(self): # ⭐️ Новий метод
        return f"Транспортний засіб {self.brand} потребує технічного огляду."

class Car(Vehicle):
    def __init__(self, brand, model, seats):
        super().__init__(brand, model)
        self.seats = seats

car = Car("Toyota", "Camry", 5)
print(car.display_info())
print(car.service_info()) # Виклик успадкованого методу

## 3. Поліморфізм

In [ ]:
class Animal:
    def speak(self):
        return "Тварина не видає звуків за замовчуванням."

class Dog(Animal):
    def speak(self):
        return "Woof!"

class Fish(Animal): # ⭐️ Клас без метода speak
    pass

animals = [Dog(), Fish()]
for a in animals:
    print(f"{type(a).__name__}: {a.speak()}")
# Пояснення: Fish використовує метод speak базового класу Animal.

## 4. Абстракція та фінальна гра 

In [ ]:
from abc import ABC, abstractmethod
from random import randint, choice

class Item(ABC): # Абстрактний клас
    def __init__(self, name: str, health=500):
        self.name = name
        self.health = health
    
    @abstractmethod
    def attack(self, another_item):
        pass

    @abstractmethod
    def boost(self):
        pass

class Sword(Item):
    def __init__(self, name, attack_power: int):
        super().__init__(name=name)
        self.__attack_power = attack_power
        self._sharp = 0
    
    def attack(self, another_item: Item):
        damage = self.__attack_power + self._sharp + randint(0, 10)
        another_item.health -= damage
        return f"⚔️ {self.name} наносить {damage} шкоди мечем."
    
    def boost(self):
        self._sharp += 5
        return f"✨ {self.name} нагострив меч (+5 до атаки)."

class Axe(Item):
    def __init__(self, name, attack_power: int):
        super().__init__(name=name)
        self.__attack_power = attack_power
    
    def attack(self, another_item: Item):
        damage = self.__attack_power + randint(0, 20)
        another_item.health -= damage
        return f"🪓 {self.name} рубає на {damage} шкоди."

    def boost(self):
        self.health += 30
        return f"🛡️ {self.name} підкріпився (+30 до HP)."

# ⭐️ НОВИЙ КЛАС: Bow
class Bow(Item):
    def __init__(self, name, attack_power: int, range_power: int):
        super().__init__(name=name)
        self.__attack_power = attack_power
        self.range_power = range_power
    
    def attack(self, another_item: Item):
        damage = self.__attack_power + randint(5, 15) + self.range_power
        another_item.health -= damage
        return f"🏹 {self.name} вистрілив на {damage} шкоди."

    def reload(self): # Унікальний метод для Bow
        self.range_power += 2
        return f"🎯 {self.name} покращив приціл (+2 до дальності)."
    
    def boost(self):
        return self.reload()

# --- Покрокова гра ---
player = Bow("Леголас", 30, 10)
enemy = choice([Sword("Артур", 40), Axe("Гімлі", 45)])

print(f"Бій: {player.name} проти {enemy.name}!")

while player.health > 0 and enemy.health > 0:
    print(f"\nТвій HP: {player.health} | Ворог HP: {enemy.health}")
    move = input("Твій хід (1 - Атака, 2 - Буст): ")
    
    if move == "1":
        print(player.attack(enemy))
    else:
        print(player.boost())
        
    if enemy.health > 0:
        # Випадковий хід ворога
        if randint(0, 1):
            print(enemy.attack(player))
        else:
            print(enemy.boost())

if player.health > 0:
    print("--- Перемога! ---")
else:
    print("--- Ти програв... ---")